# 5. SEC Annual Reports Extraction 

Given the GVKEYs and hence the CIK SEC security keys, that is inter-operable with boardex data,

We will then look at the relevant annual reports 

### Instructions

Take the metadata from the first step to get the details for the second step!!

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
sec_df = pd.read_csv("sec_gvkey_cik_us_pharma_2022_2024.csv", index_col= 0)

In [3]:
sec_df

,gvkey,conm,datadate1,datadate2,cik,source,coname,fndate,lndate,n10k,n10k_nt,n10k_a,n10q,n10q_nt,n10q_a,ndef,n8k,ntot,flag
0,1602,AMGEN INC,1983-03-31,2023-12-31,318154.0,COMPN,AMGEN INC,1994-03-28,2024-06-04,32.0,0.0,3.0,95.0,0.0,4.0,92.0,308.0,2936.0,3.0
1,1979,BALCHEM CORP -CL B,1974-12-31,2023-12-31,9326.0,COMPN,BALCHEM CORP,1995-08-11,2024-05-24,31.0,0.0,2.0,91.0,0.0,5.0,38.0,194.0,1148.0,3.0
2,2086,BAXTER INTERNATIONAL INC,1950-12-31,2023-12-31,10456.0,COMPN,BAXTER INTERNATIONAL INC,1994-03-21,2024-06-05,34.0,1.0,3.0,97.0,1.0,5.0,52.0,276.0,2321.0,3.0
3,2403,BRISTOL-MYERS SQUIBB CO,1950-12-31,2023-12-31,14272.0,COMPN,BRISTOL MYERS SQUIBB CO,1994-03-15,2024-05-09,33.0,0.0,3.0,98.0,1.0,6.0,64.0,373.0,3304.0,3.0
4,2990,IMUNON INC,1983-09-30,2023-12-31,749647.0,COMPN,"IMUNON, INC.",1996-08-07,2024-05-22,38.0,1.0,9.0,96.0,3.0,9.0,41.0,300.0,1263.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,331856,IMMUNIC INC,2018-12-31,2023-12-31,1280776.0,COMPN,"IMMUNIC, INC.",2004-02-18,2024-05-31,11.0,0.0,1.0,32.0,0.0,1.0,19.0,139.0,645.0,3.0
812,332115,ARMATA PHARMACEUTICALS INC,2018-12-31,2023-12-31,921114.0,COMPN,"ARMATA PHARMACEUTICALS, INC.",1996-02-06,2024-05-07,33.0,2.0,7.0,82.0,2.0,8.0,43.0,282.0,1062.0,3.0
813,347007,IMMUNITYBIO INC,2020-12-31,2023-12-31,1326110.0,COMPN,"IMMUNITYBIO, INC.",2005-04-29,2024-05-09,9.0,0.0,0.0,27.0,0.0,0.0,15.0,64.0,424.0,3.0
814,349972,INDAPTUS THERAPEUTICS INC,2020-12-31,2023-12-31,1857044.0,COMPN,"INDAPTUS THERAPEUTICS, INC.",2021-04-20,2024-05-22,3.0,0.0,0.0,10.0,0.0,0.0,6.0,37.0,173.0,3.0


In [4]:
cik_values = sec_df["cik"].dropna().apply(int).astype(str).drop_duplicates().to_list()

In [5]:
len(cik_values)

781

API Query

In [6]:
import config

API_KEY = config.CREDS["sec_api"]

In [7]:
from sec_api import QueryApi, ExtractorApi

In [8]:
def concatenate_to_literal_without_quotes(strings: str):
    """
    Useful function to convert to a string literal
    """

    # Convert each string in the list to be surrounded by single quotes
    quoted_strings = [f"{s}" for s in strings]
    # Join the quoted strings with a comma and space
    joined_strings = ' OR '.join(quoted_strings)
    # Format the joined strings within parentheses
    result = f"({joined_strings})"
    return result


In [9]:
concatenate_to_literal_without_quotes([1,3])

'(1 OR 3)'

In [10]:
cik_query = concatenate_to_literal_without_quotes(cik_values)

In [11]:
cik_query

'(318154 OR 9326 OR 10456 OR 14272 OR 749647 OR 109657 OR 320017 OR 704695 OR 200406 OR 59478 OR 64978 OR 310158 OR 353569 OR 1906324 OR 69499 OR 1792044 OR 73290 OR 275053 OR 78003 OR 217028 OR 101295 OR 786947 OR 730272 OR 791908 OR 1074404 OR 793524 OR 794172 OR 768408 OR 725363 OR 318306 OR 715446 OR 728387 OR 801555 OR 1116463 OR 811641 OR 812796 OR 1555280 OR 787253 OR 818033 OR 1505512 OR 1270073 OR 704562 OR 824166 OR 771856 OR 842023 OR 1293310 OR 1037643 OR 1178711 OR 1551152 OR 1177648 OR 711377 OR 1438533 OR 1388320 OR 855654 OR 1498382 OR 1347858 OR 834365 OR 1533998 OR 1117480 OR 1509261 OR 1368514 OR 1571498 OR 1476963 OR 1478242 OR 1293971 OR 1401708 OR 1439222 OR 1066745 OR 1434868 OR 1130598 OR 1403708 OR 1023024 OR 1356090 OR 1535955 OR 1434316 OR 1410939 OR 1513856 OR 1125345 OR 12239 OR 1337553 OR 1253689 OR 1723128 OR 1429560 OR 1503802 OR 1326732 OR 1384101 OR 1588972 OR 1044378 OR 1515673 OR 1346830 OR 1133869 OR 1567514 OR 1457612 OR 1485003 OR 1412486 OR 14792

1. Basic filing to get a 10K report

Write for loop if greater than 10k filings

https://sec-api.io/docs/query-api

In [13]:
# cik_values

In [14]:
queryApi = QueryApi(api_key=API_KEY)#"YOUR_API_KEY")

In [15]:
query = {
    # "query": f"cik:{cik_query} AND formType:\"10-K\"",
    # "query": "cik:1050122 AND formType:\"10-K\"",
    "query": "cik:730469 AND formType:\"10-K\"",
    # "query": "cik:(1176334 OR 1413898 OR 99780) AND formType:\"10-K\"",
    # "query": "cik:(0001050122) AND formType:\"10-K\"",
    "from": "0",
    "size": "50",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)
filings

{'total': {'value': 21, 'relation': 'eq'},
 'query': {'from': 0, 'size': 50},
 'filings': [{'ticker': 'ALO',
   'formType': '10-K',
   'accessionNo': '0000730469-08-000102',
   'cik': '730469',
   'companyNameLong': 'ALPHARMA INC (Filer)',
   'companyName': 'ALPHARMA INC',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/730469/000073046908000102/alpharmainc_10k.htm',
   'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/730469/000073046908000102/0000730469-08-000102.txt',
   'filedAt': '2008-02-27T14:45:30-05:00',
   'documentFormatFiles': [{'sequence': '1',
     'size': '664750',
     'documentUrl': 'https://www.sec.gov/Archives/edgar/data/730469/000073046908000102/alpharmainc_10k.htm',
     'description': 'ALPHARMA INC 10-K 2007',
     'type': '10-K'},
    {'sequence': '2',
     'size': '45613',
     'documentUrl': 'https://www.sec.gov/Archives/edgar/data/730469/0000730469080

Iterate over everything we need

In [16]:
def get_all_filings(cik:str):
    """
    Get the filings by CIK
    """

    query = {
        "query": f"cik:{cik} AND formType:\"10-K\"",
        "from": "0",
        "size": "50",
        "sort": [{ "filedAt": { "order": "desc" } }]
    }

    all_filings = []
    from_value = 0
    size_value = int(query['size'])

    while True:
        query['from'] = str(from_value)
        filings = queryApi.get_filings(query)
        
        # Add current batch of filings to the all_filings list
        all_filings.extend(filings['filings'])
        
        # Check if the relation is not 'gte', break the loop
        if filings['total']['relation'] != 'gte':
            break
        
        # Increment the from value by size for the next iteration
        from_value += size_value

    return all_filings

In [17]:
all_filings = get_all_filings("730469")

In [18]:
len(all_filings)

21

In [20]:
all_filings_all_dict = {cik: get_all_filings(cik) for cik in cik_values} # halved everything should take about 5 minutes instead.

In [ ]:
all_filings_all_dict # 14 s get the first 5 values in a dictionary later for details - and simplify it!!

In [ ]:
all_filings_all_dict['730469']

In [ ]:
with open("annual_report_data/all_companies_report_metadata.json", "w", encoding="utf-8") as file:

    json.dump(all_filings_all_dict, file)

In [ ]:
# run that and wait to get everything that's it!!

In [ ]:
# get dictionary with only the report link for extraction api - use this for later for extraction

# no need just separate and enrich for each json


2. Then get the url for the special section for the report that's needed

Once you get the dictionary then save the json, and then find the json details and call the corresponding api that follows

In [ ]:
from sec_api import ExtractorApi

extractorApi = ExtractorApi(API_KEY)

In [ ]:
# get the relevant filings for a particular document for each file - and save the details

all_filings_all_dict['730469']

In [ ]:
report_url = all_filings_all_dict['730469'][0]["linkToFilingDetails"] #filings["filings"][0]["linkToFilingDetails"]

In [ ]:
section_text = extractorApi.get_section(report_url, "1", "text")
print(section_text)

In [ ]:
risk_section_text = extractorApi.get_section(report_url, "1A", "text")
print(risk_section_text)

## Convert for each dictionary a single report for each for review

- Format each later for review

- Add document topics based on unique id.

In [ ]:
all_filings_all_dict['730469'][2].keys()

In [ ]:
all_filings_all_dict

In [ ]:
[{'ticker': 'ALO',
  'formType': '10-K',
  'accessionNo': '0000730469-08-000102',
  'cik': '730469',
  'companyNameLong': 'ALPHARMA INC (Filer)',
  'companyName': 'ALPHARMA INC',
  'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/730469/000073046908000102/alpharmainc_10k.htm',
  'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/730469/000073046908000102/0000730469-08-000102.txt',
  'filedAt': '2008-02-27T14:45:30-05:00',}
]

## Conclusion

Can run everything as is with the data here, and just filter the data at the end to run the relevant analytics

Text analysis

- Look at general overview and risk factors for overview structure for this.


### Data storage

cik 1 and 1A for other details later. include a filed at here.


How to filter details - put relevant graph details for all of them, must have a meaningful way to filter these.